In [21]:
import warnings
#Controle de avisos
warnings.filterwarnings("ignore")

# Installing CrewAI and CrewAI Tools

In [22]:
!python -m pip install crewai

In [23]:
!pip install crewai_tools

# Importing Necessary Libraries


In [24]:
from crewai import Agent, Task, Crew
import os
from IPython.display import Markdown
from google.colab import userdata


# Getting the API Key in a safe way
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# Setting the model to use for the CrewAI Agents
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Importing new tools from crewai_tools and spacy library


In [25]:
# SerperDevTool is a tool likely used for searching the web or accessing specific APIs using the Serper search engine.
# ScrapeWebsiteTool is designed to extract information from websites by scraping their content.
# WebsiteSearchTool likely allows agents to search a specific website for information.
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool
import spacy  # Library for NLP analysis

In [26]:
# Load language model
nlp = spacy.load("en_core_web_sm")

# Creating the Agents

In [27]:
# Agent 1: Collects patient information
# Creates the agent using the Agent class, "role" defines its role, the "goal"
# sets the agent's objective and guides its actions, and the "backstory" is the
# backgroun and add context
collector = Agent(
    role="Symptom Collector",
    goal="Gather detailed information about the patient's symptoms",
    backstory=(
        "You are a virtual assistant responsible for collecting patient information "
        "in a clear and organized manner. Your goal is to ensure no important detail "
        "is overlooked and that the information is correctly passed on for analysis."
    ),
    allow_delegation=False, # The agent will not use help from other agents
    verbose=True # Defines that it will provide detailed log and messages
)

In [28]:
# Agent 2: Analyzes symptoms
# Creates the agent using the Agent class, "role" defines its role, the "goal"
# sets the agent's objective and guides its actions, and the "backstory" is the
# backgroun and add context
analyzer = Agent(
    role="Medical Analyst",
    goal="Identify and interpret the symptoms described by the patient",
    backstory=(
        "You are an advanced AI system specialized in medical analysis. "
        "Your job is to interpret the symptoms described by patients and identify "
        "possible medical conditions, assisting in triage."
    ),
    allow_delegation=False, # The agent will not use help from other agents
    verbose=True # Defines that it will provide detailed log and messages
)

In [29]:
# Agent 3: Prioritizes treatment
# Creates the agent using the Agent class, "role" defines its role, the "goal"
# sets the agent's objective and guides its actions, and the "backstory" is the
# backgroun and add context
prioritizer = Agent(
    role="Treatment Prioritizer",
    goal="Classify the urgency of treatment based on the presented symptoms",
    backstory=(
        "You are a specialist in medical triage, responsible for classifying "
        "cases into different priority levels. Your goal is to ensure that "
        "the most critical patients receive urgent care."
    ),
    allow_delegation=False, # The agent will not use help from other agents
    verbose=True # Defines that it will provide detailed log and messages
)

# Creating the tasks

In [30]:
# Defining a task using the "task" class from crewai
# "description" provides the context and instructions for the task
# "expected_outputs" outlines the desired outcome
# "tools" defines the tools tath the agent can use for that task
# "agent" assigns the task to a agent

# These Agent will collect the patient data and create an organized list with this
collect_symptoms_task = Task(
    description=(
        "A new patient has arrived and needs to provide details about their symptoms. "
        "You must collect the following information: name, age, and a clear description "
        "of the symptoms presented. Ensure that the information is well-organized "
        "and complete."
    ),
    expected_output=(
        "A structured dataset containing the patient's name, age, and a list of symptoms "
        "described by them. No relevant information should be missing."
    ),
    agent=collector
)

In [31]:
# Defining a task using the "task" class from crewai
# "description" provides the context and instructions for the task
# "expected_outputs" outlines the desired outcome
# "tools" defines the tools tath the agent can use for that task
# "agent" assigns the task to a agent

# These Agent will analize the data received from the previous Agent with nlp
analyze_symptoms_task = Task(
    description=(
        "The patient has provided a description of their symptoms. Your task is to process "
        "this description using NLP to extract relevant symptoms. Ensure that "
        "the extracted symptoms accurately represent the patient's condition."
    ),
    expected_output=(
        "A list containing the symptoms identified from the patient's description, "
        "using natural language processing."
    ),
    tools=[SerperDevTool(), ScrapeWebsiteTool(), WebsiteSearchTool()],
    agent=analyzer
)

In [32]:
# Defining a task using the "task" class from crewai
# "description" provides the context and instructions for the task
# "expected_outputs" outlines the desired outcome
# "tools" defines the tools tath the agent can use for that task
# "agent" assigns the task to a agent

# These Agent will define a classification for each case, betwen emergency, urgent or elective
prioritize_case_task = Task(
    description=(
        "Based on the identified symptoms, you must determine the patient's treatment priority. "
        "Cases can be classified as 'Emergency,' 'Urgent,' or 'Elective.' "
        "More severe symptoms should be treated with higher priority."
    ),
    expected_output=(
        "A priority classification for the patient's treatment, ensuring that "
        "the most critical cases receive immediate attention."
    ),
    agent=prioritizer
)

# Creating functions

In [33]:
# The function collects the patient data
def collect_patient_info():

    return {
        "name": input("What is your name? "),
        "age": int(input("What is your age? ")),
        "symptoms": input("Describe your symptoms: "),
    }

In [34]:
# The function analyze the data using nlp by the spacy library and the data collected by the previous Agent
def analyze_patient(patient_data):
    doc = nlp(patient_data["symptoms"])
    symptoms = [token.text for token in doc.ents if token.label_ == "SYMPTOM"]
    return symptoms if symptoms else ["No symptoms identified"]

In [35]:
# The function will define the level of the symptoms
def prioritize_case(symptoms):
    emergency_keywords = ["severe pain", "shortness of breath", "fainting"]
    urgent_keywords = ["high fever", "frequent vomiting", "infection"]

    if any(symptom in emergency_keywords for symptom in symptoms):
        return "Emergency"
    elif any(symptom in urgent_keywords for symptom in symptoms):
        return "Urgent"
    else:
        return "Elective"

# Creating the Crew

In [36]:
# Creating the crew from the Crew class form crewai
# "agents" assing the agents to be used
# "tasks" defines the sequence of tasks to be executed in the correct order
crew = Crew(
    agents=[collector, analyzer, prioritizer],
    tasks=[collect_symptoms_task, analyze_symptoms_task, prioritize_case_task],
    verbose=True,  # Provides detailed logs
    memory=True  # Allows agents to use memory to improve results
)

# Starting the Crew

In [40]:
# begins the actions calling the "kickoff" function of the "crew" object providing
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: fdecb6ad-ba4e-45ab-8ea8-b152765b72c1                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 95280bdb-ed73-456e-af03-e9b2d15420a6
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 95280bdb-ed73-456e-af03-e9b2d15420a6
       Status: Executing Task...
    └── 🤖 Agent: Symptom Collector
            Status: In Progress

# Agent: Symptom Collector
## Task: A new patient has arrived and needs to provide details about their symptoms. You must collect the following information: name, age, and a clear description of the symptoms presented. Ensure that the information is well-organized and complete.


🚀 Crew: crew
└── 📋 Task: 95280bdb-ed73-456e-af03-e9b2d15420a6
       Status: Executing Task...
    └── 🤖 Agent: Symptom Collector
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Treatment Prioritizer
## Final Answer: 
Patient Name: John Smith
Age: 45
Symptoms:
- Symptom 1: Chest pain radiating to left arm, sweating
- Symptom 2: Shortness of breath, dizziness
- Symptom 3: Nausea, vomiting

Entities:
- Chest pain radiating to left arm, sweating, shortness of breath, dizziness, nausea, and vomiting (Symptoms): List of symptoms presented by the patient during the evaluation for treatment priority classification.
- Chest pain radiating to left arm, sweating (Symptom 1): This combination of symptoms is highly indicative of a potential heart attack, especially when accompanied by shortness of breath.
- Shortness of breath, dizziness (Symptom 2): These symptoms can also be associated with cardiac issues and further support the urgency of the situation.
- Nausea, vomiting (Symptom 3): While these symptoms can have various causes, when combined with the other symptoms, they raise concerns for a cardiac event or other serious medical condition.

Based on the p

🚀 Crew: crew
├── 📋 Task: 95280bdb-ed73-456e-af03-e9b2d15420a6
│      Assigned to: Symptom Collector
│      Status: ✅ Completed
│   └── 🤖 Agent: Symptom Collector
│           Status: ✅ Completed
├── 📋 Task: 0cdae356-97da-49ae-97e1-a994c5e6dc3b
│      Assigned to: Medical Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Medical Analyst
│           Status: ✅ Completed
│       ├── 🔧 Failed Search the internet with Serper (3)
│       └── 🔧 Used Search in a specific website (3)
└── 📋 Task: 1d0541cf-efea-47a4-ba4f-543d92ce190c
       Status: Executing Task...
    └── 🤖 Agent: Treatment Prioritizer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 95280bdb-ed73-456e-af03-e9b2d15420a6
│      Assigned to: Symptom Collector
│      Status: ✅ Completed
│   └── 🤖 Agent: Symptom Collector
│           Status: ✅ Completed
├── 📋 Task: 0cdae356-97da-49ae-97e1-a994c5e6dc3b
│      Assigned to: Medical Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Medical Analyst
│           Status: ✅ Completed
│       ├── 🔧 Failed Search the internet with Serper (3)
│       └── 🔧 Used Search in a specific website (3)
└── 📋 Task: 1d0541cf-efea-47a4-ba4f-543d92ce190c
       Assigned to: Treatment Prioritizer
       Status: ✅ Completed
    └── 🤖 Agent: Treatment Prioritizer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1d0541cf-efea-47a4-ba4f-543d92ce190c                                                                     │
│  Agent: Treatment Prioritizer                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: fdecb6ad-ba4e-45ab-8ea8-b152765b72c1                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Displaying the Result

In [41]:
# the markdown shows the clean output
Markdown(result.raw)

Patient Name: John Smith
Age: 45
Symptoms:
- Symptom 1: Chest pain radiating to left arm, sweating
- Symptom 2: Shortness of breath, dizziness
- Symptom 3: Nausea, vomiting

Entities:
- Chest pain radiating to left arm, sweating, shortness of breath, dizziness, nausea, and vomiting (Symptoms): List of symptoms presented by the patient during the evaluation for treatment priority classification.
- Chest pain radiating to left arm, sweating (Symptom 1): This combination of symptoms is highly indicative of a potential heart attack, especially when accompanied by shortness of breath.
- Shortness of breath, dizziness (Symptom 2): These symptoms can also be associated with cardiac issues and further support the urgency of the situation.
- Nausea, vomiting (Symptom 3): While these symptoms can have various causes, when combined with the other symptoms, they raise concerns for a cardiac event or other serious medical condition.

Based on the presented symptoms, which strongly suggest a possible heart attack, the treatment priority for John Smith should be classified as **Emergency**. Immediate medical attention is required to assess and manage the potential cardiac emergency to ensure the best possible outcome for the patient.

This classification is crucial to prioritize the most critical cases and provide timely and appropriate care to the patient, considering the severity and potential life-threatening nature of the symptoms presented.